In [1]:
import numpy as np
from IPython.display import display, Image
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import pandas as pd
from hmmlearn import hmm
import numpy as np
import seaborn as sns
from scipy.stats import norm
import itertools
import matplotlib.pyplot as plt
from hmmlearn.hmm import GMMHMM
from  sklearn.metrics import mean_squared_error as mse
from astropy.io import fits
import matplotlib.pyplot as plt
import os, sys
dirpath = os.getcwd().split("code")[0]+"code/"
sys.path.append(dirpath)
from pre_process import clean_LC

def plot_state_matrix(cm, states,title='Transition Probabilities',cmap=plt.cm.Blues):
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(states))
    plt.xticks(tick_marks, states)
    plt.yticks(tick_marks, states)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], '.3f'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > cm.max() / 2. else "black")
    plt.ylabel('Origin state')
    plt.xlabel('Destination state')
    plt.tight_layout()

    
def plot_model_and_data(plot_lc, model, p_n=2000, real_lc=[] ):
    X_state = model.predict(plot_lc) #predice el estado, no el valor 
    X_state_mean = model.means_[X_state][:,0]
    X_state_cov = np.sqrt(model.covars_[X_state][:,0,0])
    if len(real_lc) != 0:
        if len(plot_lc) != len(real_lc):
            mask_nan = np.isnan(real_lc)
            aux = np.tile(np.nan, len(real_lc) )
            
            aux_lc = aux.copy()
            aux_lc[~mask_nan] = np.squeeze(plot_lc)
            plot_lc = aux_lc
            
            aux_X_state_mean = aux.copy()
            aux_X_state_mean[~mask_nan] = X_state_mean
            X_state_mean = aux_X_state_mean
            
            aux_X_state_cov = aux.copy()
            aux_X_state_cov[~mask_nan] = X_state_cov
            X_state_cov = aux_X_state_cov
    plt.figure(figsize=(14,6))
    plt.plot(plot_lc,'r.-',label="Data")
    plt.fill_between(np.arange(len(plot_lc)),
                     X_state_mean-1.96*X_state_cov,
                     X_state_mean+1.96*X_state_cov) 
    plt.plot(X_state_mean,'g*-',label="Prediccion de la media") 
    plt.legend()
    plt.title("prediccion de markov en curva")
    plt.show()
    
    plt.figure(figsize=(14,6))
    plt.plot(plot_lc[:p_n],'r.-',label="Data")
    plt.fill_between(np.arange(p_n),
                     X_state_mean[:p_n]-1.96*X_state_cov[:p_n],
                     X_state_mean[:p_n]+1.96*X_state_cov[:p_n]) 
    plt.plot(X_state_mean[:p_n],'g*-',label="Prediccion de la media") 
    plt.legend()
    plt.title("prediccion de markov zoom en curva")
    plt.show()
    
    
#older = "/work/work_teamEXOPLANET/KOI_LC/"
df_meta=pd.read_csv('../../KOI_Data/kepler_dataset.csv')
df_meta.head()

,Kepler ID,KOI Name,Kepler Name,KOI Number,Kepler Disposition,NExScI Disposition,RA (J2000),RA Error,Dec (J2000),Dec Error,...,Contamination Flag,Insolation Flux,Insolation Flux err1,Insolation Flux err2,Fitted Stellar Density,Fitted Stellar Density err1,Fitted Stellar Density err2,Planetary Fit,Disposition Score,File Name
0,757450,K00889.01,Kepler-75 b,889.01,CANDIDATE,CONFIRMED,19 24 33.024,0.0,+36 34 38.57,0.0,...,0,83.21,16.69,-13.87,11.62061,0.20683,-0.40258,LS+MCMC,0.999,kplr000757450-20160128150956_dvt.fits
1,892772,K01009.01,NaN,1009.01,FALSE POSITIVE,FALSE POSITIVE,19 24 36.811,0.0,+36 40 43.83,0.0,...,0,119.70,48.69,-36.91,0.88510,0.10080,-0.39457,LS+MCMC,0.000,kplr000892772-20160128150956_dvt.fits
2,1025986,K07621.01,NaN,7621.01,FALSE POSITIVE,FALSE POSITIVE,19 24 08.086,0.0,+36 46 15.75,0.0,...,0,2.00,0.92,-0.76,33.55994,16.58298,-31.53180,LS+MCMC,0.000,kplr001025986-20160128150956_dvt.fits
3,1026032,K06252.01,NaN,6252.01,FALSE POSITIVE,FALSE POSITIVE,19 24 10.577,0.0,+36 43 45.38,0.0,...,0,98.60,52.46,-21.18,0.73874,0.00021,-0.00735,LS+MCMC,0.000,kplr001026032-20160128150956_dvt.fits
4,1026957,K00958.01,NaN,958.01,CANDIDATE,CANDIDATE,19 25 01.078,0.0,+36 44 37.00,0.0,...,0,13.18,3.00,-1.98,102.86534,5.47239,-30.25813,LS+MCMC,1.000,kplr001026957-20160128150956_dvt.fits


In [2]:
! ls '/work/work_teamEXOPLANET/KOI_LC/cleaned/'

LC_kepler_processed.npy  LC_kepler_processed_old.npy


In [5]:
cand=df_meta.loc[df_meta['NExScI Disposition']=='CANDIDATE']
cand

,Kepler ID,KOI Name,Kepler Name,KOI Number,Kepler Disposition,NExScI Disposition,RA (J2000),RA Error,Dec (J2000),Dec Error,...,Contamination Flag,Insolation Flux,Insolation Flux err1,Insolation Flux err2,Fitted Stellar Density,Fitted Stellar Density err1,Fitted Stellar Density err2,Planetary Fit,Disposition Score,File Name
4,1026957,K00958.01,NaN,958.01,CANDIDATE,CANDIDATE,19 25 01.078,0.0,+36 44 37.00,0.0,...,0,13.18,3.00,-1.98,102.86534,5.47239,-30.25813,LS+MCMC,1.000,kplr001026957-20160128150956_dvt.fits
5,1161345,K00984.01,NaN,984.01,CANDIDATE,CANDIDATE,19 24 11.712,0.0,+36 50 23.58,0.0,...,0,186.31,56.73,-28.39,8.16471,4.69586,-1.63188,LS+MCMC,0.711,kplr001161345-20160128150956_dvt.fits
8,1431122,K00994.01,NaN,994.01,CANDIDATE,CANDIDATE,19 24 34.198,0.0,+37 03 40.76,0.0,...,0,283.62,100.01,-78.24,0.05666,3.20152,-0.06620,LS+MCMC,0.990,kplr001431122-20160128150956_dvt.fits
11,1432789,K00992.02,NaN,992.02,CANDIDATE,CANDIDATE,19 25 59.861,0.0,+37 03 33.41,0.0,...,0,499.41,210.40,-153.04,0.58469,0.04511,-0.31824,LS+MCMC,0.961,kplr001432789-20160128150956_dvt.fits
18,1575873,K04359.01,NaN,4359.01,CANDIDATE,CANDIDATE,19 27 42.326,0.0,+37 06 04.65,0.0,...,0,30567.67,18880.37,-17052.93,2.25862,15.02837,-1.46534,LS+MCMC,1.000,kplr001575873-20160128150956_dvt.fits
19,1717722,K03145.01,NaN,3145.01,CANDIDATE,CANDIDATE,19 22 28.315,0.0,+37 17 20.72,0.0,...,0,95.61,38.11,-21.67,2.61043,4.63396,-2.21540,LS+MCMC,1.000,kplr001717722-20160128150956_dvt.fits
20,1717722,K03145.02,NaN,3145.02,CANDIDATE,CANDIDATE,19 22 28.315,0.0,+37 17 20.72,0.0,...,0,737.36,293.96,-167.10,3.17024,0.35594,-1.81104,LS+MCMC,0.858,kplr001717722-20160128150956_dvt.fits
24,1718958,K04053.01,NaN,4053.01,CANDIDATE,CANDIDATE,19 23 40.642,0.0,+37 17 13.81,0.0,...,0,1968.38,836.24,-503.00,1.95904,1.98658,-1.91843,LS+MCMC,0.985,kplr001718958-20160128150956_dvt.fits
25,1721157,K04644.01,NaN,4644.01,CANDIDATE,CANDIDATE,19 25 49.762,0.0,+37 12 24.63,0.0,...,0,206.55,91.39,-46.68,6.48403,5.43333,-3.39902,LS+MCMC,0.970,kplr001721157-20160128150956_dvt.fits
27,1723078,K07624.01,NaN,7624.01,CANDIDATE,CANDIDATE,19 27 28.322,0.0,+37 16 11.56,0.0,...,0,12.36,5.55,-8.42,4.04249,14.39050,-3.71379,LS+MCMC,0.246,kplr001723078-20160128150956_dvt.fits


In [6]:
index_ca=list(cand.index)
len(index_ca)

1797

In [7]:
kplr_ids=np.asarray(df_meta['Kepler ID'].values)[[index_ca]]
kplr_ids.shape

/users/mbugueno/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


(1797,)

In [8]:
kois_name=np.asarray(df_meta['KOI Name'].values)[[index_ca]]
kois_name.shape

/users/mbugueno/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


(1797,)

### recuperando FP 20 estados

In [9]:
our_detrends=np.load('/work/work_teamEXOPLANET/KOI_LC/cleaned/LC_kepler_processed.npy')
print (our_detrends.shape)
our_detrends=our_detrends[index_ca]
our_detrends.shape

(8054, 71427)


(1797, 71427)

In [10]:
folder_marg='/work/work_teamEXOPLANET/MTF/'

In [ ]:
import os

i=0
for lc_our_detrend in our_detrends:  
    recuperado=False
    name_kepler=kois_name[i]
    id_kepler=kplr_ids[i]    
    try:
        a=np.load(folder_marg+'candidatos_20/'+str(name_kepler)+'.npy')
        print ("matriz ", name_kepler, 'recuperada previamente')
        i+=1
        continue
    
    except:         
        print ("Calculando matriz para ", name_kepler, "pues tenia kepler id",id_kepler, "con i:",i)
        lc_wind_nan = []
        lengths = []
        sublist = []
        for value in lc_our_detrend:
            if np.isnan(value) and len(sublist) != 0:
                lc_wind_nan.append(np.asarray(sublist))
                lengths.append(len(sublist))
                sublist = []
            elif np.isnan(value) and len(sublist) == 0:
                continue
            else: 
                sublist.append(value) 
        if len(sublist) != 0:      
            lc_wind_nan.append(np.asarray(sublist))
            lengths.append(len(sublist))
        lc_wind_nan = np.asarray(lc_wind_nan)   
        lengths = np.asarray(lengths)   

        lc_wind_nan = np.concatenate(lc_wind_nan).reshape(-1,1)
        lc_wind_nan = lc_wind_nan/np.abs(np.min(lc_wind_nan)) 
        n_sta=20
        markov_model = hmm.GaussianHMM(n_components=n_sta, n_iter=50)
        markov_model.fit(lc_wind_nan , lengths)
        np.save(folder_marg+'candidatos_20/'+str(name_kepler)+'.npy', markov_model.transmat_)
        i+=1


Calculando matriz para  K00958.01 pues tenia kepler id 1026957 con i: 0
Calculando matriz para  K00984.01 pues tenia kepler id 1161345 con i: 1
Calculando matriz para  K00994.01 pues tenia kepler id 1431122 con i: 2
Calculando matriz para  K00992.02 pues tenia kepler id 1432789 con i: 3
Calculando matriz para  K04359.01 pues tenia kepler id 1575873 con i: 4
Calculando matriz para  K03145.01 pues tenia kepler id 1717722 con i: 5
Calculando matriz para  K03145.02 pues tenia kepler id 1717722 con i: 6
Calculando matriz para  K04053.01 pues tenia kepler id 1718958 con i: 7
Calculando matriz para  K04644.01 pues tenia kepler id 1721157 con i: 8
Calculando matriz para  K07624.01 pues tenia kepler id 1723078 con i: 9
Calculando matriz para  K04212.02 pues tenia kepler id 1724719 con i: 10
Calculando matriz para  K02538.01 pues tenia kepler id 1849702 con i: 11
Calculando matriz para  K04926.01 pues tenia kepler id 1865864 con i: 12
Calculando matriz para  K02351.01 pues tenia kepler id 187282

Calculando matriz para  K02288.01 pues tenia kepler id 3440978 con i: 113
Calculando matriz para  K02043.01 pues tenia kepler id 3448130 con i: 114
Calculando matriz para  K04205.01 pues tenia kepler id 3450040 con i: 115
Calculando matriz para  K00802.01 pues tenia kepler id 3453214 con i: 116
Calculando matriz para  K03340.01 pues tenia kepler id 3529290 con i: 117
Calculando matriz para  K03340.02 pues tenia kepler id 3529290 con i: 118
Calculando matriz para  K04626.01 pues tenia kepler id 3539231 con i: 119
Calculando matriz para  K04987.01 pues tenia kepler id 3542117 con i: 120
Calculando matriz para  K03279.01 pues tenia kepler id 3544640 con i: 121
Calculando matriz para  K02755.01 pues tenia kepler id 3545135 con i: 122
Calculando matriz para  K00366.01 pues tenia kepler id 3545478 con i: 123
Calculando matriz para  K06342.01 pues tenia kepler id 3547178 con i: 124
Calculando matriz para  K04535.01 pues tenia kepler id 3547760 con i: 125
Calculando matriz para  K02194.03 pues

Calculando matriz para  K07693.01 pues tenia kepler id 4384909 con i: 224
Calculando matriz para  K02942.01 pues tenia kepler id 4385148 con i: 225
Calculando matriz para  K02942.02 pues tenia kepler id 4385148 con i: 226
Calculando matriz para  K08246.01 pues tenia kepler id 4446859 con i: 227
Calculando matriz para  K00625.01 pues tenia kepler id 4449034 con i: 228
Calculando matriz para  K03168.01 pues tenia kepler id 4450844 con i: 229
Calculando matriz para  K02303.02 pues tenia kepler id 4458082 con i: 230
Calculando matriz para  K01338.02 pues tenia kepler id 4466677 con i: 231
Calculando matriz para  K01338.03 pues tenia kepler id 4466677 con i: 232
Calculando matriz para  K01656.01 pues tenia kepler id 4471747 con i: 233
Calculando matriz para  K03878.02 pues tenia kepler id 4472818 con i: 234
Calculando matriz para  K02229.01 pues tenia kepler id 4473226 con i: 235
Calculando matriz para  K04452.01 pues tenia kepler id 4474462 con i: 236
Calculando matriz para  K05067.01 pues